# Анализ нормализованных событий SIEM с использованием AutoGen


Обработка происходит в несколько этапов:

1. Чтение с диска, извлечение, разделение и ограничение количества событий

2. Обработка через сопоставление с MITRE ATT&CK и проверку IP и Domain в VirusTotal

3. Структурированный парсинг, объединение и агрегация результатов

4. Финальное саммари

In [4]:
# Установим бибилиотки, если еще не установили их из requirements.txt
!pip install \
    loguru==0.7.3 \
    autogen-agentchat==0.5.3 \
    autogen-ext[openai,azure,semantic-kernel-mistralai]==0.5.3 \
    qdrant-client==1.13.3 \
    langchain==0.3.23 \
    langchain-qdrant==0.2.0 \
    langchain-text-splitters==0.3.8 \
    langchain-openai==0.3.14 \
    vt-py==0.20.0 \
    nest_asyncio==1.6.0 \
    python-dotenv==1.1.0 \
    langchain-mistralai==0.2.10

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/901.8 kB ? eta -:--:--
   ---------------------------------------- 901.8/901.8 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/27.9 MB ? eta -:--:--
   --- ------------------------------------ 2.1/27.9 MB 10.7 MB/s eta 0:00:03
   ------ --------------------------------- 4.2/27.9 MB 10.1 MB/s eta 0:00:03
   -------- ------------------------------- 6.0/27.9 MB 10.0 MB/s eta 0:00:03
   ----------- ---------------------------- 8.1/27.9 MB 9.9 MB/s eta 0:00:03
   -------------- ------------------------- 10.0/27.9 MB 9.5 MB/s eta 0:00:02
   ---------------- ----------------------- 11.3/27.9 MB 9.3 MB/s eta 0:00:02
   ------------------ --------------------- 

  DEPRECATION: Building 'pybars4' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pybars4'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'PyMeta3' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'PyMeta3'. Discussion can be found at https://github.com/pypa/pip/issues/6334


## Закомментированнный код ниже - опционален, нужен для трейсинга в Langfuse

In [1]:
# !pip install opentelemetry-sdk==1.32.1 opentelemetry-exporter-otlp==1.32.1 openlit==1.33.19

  Using cached opentelemetry_sdk-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp-1.32.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached openlit-1.33.19-py3-none-any.whl.metadata (23 kB)
  Using cached opentelemetry_api-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_semantic_conventions-0.53b1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.32.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.32.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached anthropic-0.42.0-py3-none-any.whl.metadata (23 kB)
  Using cached opentelemetry_instrumentation-0.48b0-py3-none-any.whl.metadata (6.1 kB)
  Using cached schedule-1.2.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached xmltodict-0.13.0-py2.py3-none-any.whl.metadata (7.7 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached opentelemetry_exporter_otlp_pr

In [ ]:
# import base64
# import os

# LANGFUSE_AUTH = base64.b64encode(
#     f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
# ).decode()

# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel"
# os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [ ]:
# from opentelemetry.sdk.trace import TracerProvider
# from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
# from opentelemetry.sdk.trace.export import SimpleSpanProcessor
 
# trace_provider = TracerProvider()
# trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))
 
# # Sets the global default tracer provider
# from opentelemetry import trace
# trace.set_tracer_provider(trace_provider)

# # Creates a tracer from the global tracer provider
# tracer = trace.get_tracer(__name__)

In [ ]:
# import openlit
 
# # Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
# openlit.init(tracer=tracer, disable_batch=True)

In [1]:
import json
import os
import sys
import time
from loguru import logger
from typing import List, Dict, Any
from typing_extensions import Annotated
from dotenv import load_dotenv
import nest_asyncio
import traceback


load_dotenv()

logger.remove()

log_format = (
    "<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green> | "
    "<level>{level: <8}</level> | "
    "<cyan>{extra[name]}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - <level>{message}</level>"
)

logger.add(
    sys.stderr,
    format=log_format,
    level="DEBUG",  # Log DEBUG and above to console during development
    colorize=True
)

logger.configure(extra={"name": "SIEM_AUTOGEN_ANALYSIS"})

nest_asyncio.apply()

## 1. Читаем файлы с диска
Для начала загрузим файлы из JSON файла

In [2]:
def load_siem_events(file_path: str) -> List[Dict[str, Any]]:
    """Load normalized SIEM events from a JSON file"""
    with open(file_path, 'r') as f:
        events = json.load(f)
    return events

# siem_events_path = "../local-files/main_dump.json"
siem_events_path = "../local-files/main_dump_short.json"

siem_events = load_siem_events(siem_events_path)
# siem_events = siem_events['ProcessTree'] + siem_events['RelatedEvents']

logger.info(f"Loaded {len(siem_events)} SIEM events")

2025-05-21 16:41:57.829 | INFO     | SIEM_AUTOGEN_ANALYSIS:<module>:13 - Loaded 7 SIEM events


## 2. Фильтруем поля, если нужно
После фильтрации ограничим количество событий, для более быстрой демонстрации

In [3]:
def extract_key_info(event: Dict[str, Any]) -> Dict[str, Any]:
    """Extract key information from a SIEM event"""
    key_info = {
        "id": event.get("id", ""),
        "time": event.get("time", ""),
        "body": event.get("body", ""),
        "subject": event.get("subject", ""),
        "action": event.get("action", ""),
        "status": event.get("status", ""),
        "src_ip": event.get("src.ip", ""),
        "dst_hostname": event.get("dst.hostname", ""),
        "event_src_title": event.get("event_src.title", ""),
        "category_high": event.get("category.high", ""),
        "category_generic": event.get("category.generic", ""),
        "category_low": event.get("category.low", "")
    }
    return key_info


def process_events(events: List[Dict[str, Any]], limit: int = 10) -> List[Dict[str, Any]]:
    """Process events and limit the number"""
    processed_events = [extract_key_info(event) for event in events]
    return processed_events[:limit]


processed_events = siem_events
# process_events(siem_events, limit=10)
logger.info(f"Processing {len(processed_events)} events")

if processed_events:
    logger.info("\nSample event:")
    logger.info(json.dumps(processed_events[0], indent=2))

2025-05-21 16:41:59.836 | INFO     | SIEM_AUTOGEN_ANALYSIS:<module>:28 - Processing 7 events
2025-05-21 16:41:59.837 | INFO     | SIEM_AUTOGEN_ANALYSIS:<module>:31 - 
Sample event:
2025-05-21 16:41:59.838 | INFO     | SIEM_AUTOGEN_ANALYSIS:<module>:32 - {
  "action": "detect",
  "agent_id": "e24b4795-36be-4043-b8f5-00a480c8428d",
  "asset_ids": [
    "1b5edacf-03c0-0001-0000-000000000061",
    "1b5edb23-4c40-0001-0000-000000000075",
    "1b605601-12c0-0001-0000-000000000253"
  ],
  "body": "{\"Event\":{\"xmlns\":\"http://schemas.microsoft.com/win/2004/08/events/event\",\"System\":{\"Provider\":{\"Name\":\"Microsoft-Windows-Sysmon\",\"Guid\":\"{5770385f-c22a-43e0-bf4c-06f5698ffbd9}\"},\"EventID\":\"3\",\"Version\":\"5\",\"Level\":\"4\",\"Task\":\"3\",\"Opcode\":\"0\",\"Keywords\":\"0x8000000000000000\",\"TimeCreated\":{\"SystemTime\":\"2024-11-28T14:40:05.5183495Z\"},\"EventRecordID\":\"2451296\",\"Execution\":{\"ProcessID\":\"3472\",\"ThreadID\":\"4980\"},\"Channel\":\"Microsoft-Window

## 3. Загружаем данные MITRE ATT&CK

Также из подготовленного и очищенного JSON файла

In [24]:
def load_mitre_data(file_path: str) -> List[Dict[str, Any]]:
    """Load MITRE ATT&CK data from a JSON file"""
    with open(file_path, 'r') as f:
        mitre_data = json.load(f)
    return mitre_data


mitre_data_path = "../local-files/cleaned_mitre_attack_data_short.json"
mitre_data = load_mitre_data(mitre_data_path)
logger.info(f"Loaded {len(mitre_data)} MITRE ATT&CK techniques")

2025-05-19 22:48:50.581 | INFO     | SIEM_AUTOGEN_ANALYSIS:<module>:10 - Loaded 1 MITRE ATT&CK techniques


## 4. Инициализируем модель эмбеддингов

Модель будет использоваться для векторизации описаний из MITRE и для векторного поиска

In [4]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_mistralai import MistralAIEmbeddings


def setup_vector_store(oai_embeddings_client, collection_name='mitre_demo', vector_size=1024):
    qdrant_client = QdrantClient(
        url=os.environ.get("QDRANT_URL", "http://localhost:6333"),
        api_key=os.environ.get("QDRANT_API_KEY", None)
    )

    collection_exists = qdrant_client.collection_exists(collection_name)
    if not collection_exists:
        logger.info(f"Creating new collection: {collection_name}")
        qdrant_client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(
                size=vector_size, distance=Distance.COSINE
            )
        )
    else:
        logger.info(f"Collection {collection_name} already exists")

    return QdrantVectorStore(
        client=qdrant_client,
        collection_name=collection_name,
        embedding=oai_embeddings_client,
        retrieval_mode=RetrievalMode.DENSE,
    )

def prepare_documents(data: List[Dict[str, Any]], text_splitter) -> List[Document]:
    """
    Convert MITRE data entries to Document objects and split them into chunks.
    For each entry, the description is used as content, and other fields are stored as metadata.
    """
    documents = []
    
    for entry in data:
        if 'description' in entry and entry['description']:
            metadata = {
                'id': entry.get('id', ''),
                'name': entry.get('name', ''),
                'kill_chain_phases': entry.get('kill_chain_phases', []),
                'external_references': entry.get('external_references', [])
            }
            
            doc = Document(page_content=entry['description'], metadata=metadata)
            documents.append(doc)
    
    chunks = text_splitter.split_documents(documents)
    logger.info(f"Created {len(chunks)} document chunks from {len(documents)} original documents")
    
    return chunks

## 5. Создаем векторное хранилище на базе Qdrant

В него сложим данные описаний MITRE

In [5]:
embed_model_name="mistral-embed"
chunk_size=1000
chunk_overlap=100

file_path = "../local-files/cleaned_mitre_attack_data.json"

with open(file_path, 'r') as file:
    data = json.load(file)

embeddings_client = MistralAIEmbeddings(
    model=embed_model_name
)

vector_store = setup_vector_store(embeddings_client)

c:\Users\apekhterev\.conda\envs\agents-wshp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\apekhterev\.conda\envs\agents-wshp\lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(
2025-05-21 16:42:19.270 | INFO     | SIEM_AUTOGEN_ANALYSIS:setup_vector_store:26 - Collection mitre_demo already exists


In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

chunks = prepare_documents(data, text_splitter)


logger.info(f"Adding {len(chunks)} document chunks to vector store")
await vector_store.aadd_documents(chunks, batch_size=32)

logger.info("MITRE data processing completed successfully")

2025-05-19 20:19:30.466 | INFO     | SIEM_AUTOGEN_ANALYSIS:prepare_documents:55 - Created 1648 document chunks from 799 original documents
2025-05-19 20:19:30.470 | INFO     | SIEM_AUTOGEN_ANALYSIS:<module>:9 - Adding 1648 document chunks to vector store
2025-05-19 20:38:52.490 | INFO     | SIEM_AUTOGEN_ANALYSIS:<module>:12 - MITRE data processing completed successfully


## 6. Засетапим ллм для агентов

А также самих агентов - пропишем роли в системные промпты

In [ ]:
# from autogen_ext.models.openai import OpenAIChatCompletionClient
# from autogen_core.models import ModelFamily


# tool_calling_model_client = OpenAIChatCompletionClient(
#     model='openai/gpt-4.1-nano',
#     api_key=os.environ.get('OPENAI_API_KEY'),
#     base_url = os.environ.get('OPENAI_API_BASE'),
#     model_info={
#         "vision": False,
#         "function_calling": True,
#         "json_output": True,
#         "family": ModelFamily.UNKNOWN,
#         "structured_output": True,
#     },
#     temperature = 0.1
# )

# small_model_client = OpenAIChatCompletionClient(
#     model='openai/gpt-4.1-nano',
#     api_key=os.environ.get('OPENAI_API_KEY'),
#     base_url = os.environ.get('OPENAI_API_BASE'),
#     model_info={
#         "vision": True,
#         "function_calling": False,
#         "json_output": True,
#         "family": ModelFamily.UNKNOWN,
#         "structured_output": True,
#     },
#     temperature = 0.1
# )

In [6]:
from autogen_core.models import ModelFamily
from autogen_ext.models.semantic_kernel import SKChatCompletionAdapter
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.mistral_ai import MistralAIChatCompletion, MistralAIChatPromptExecutionSettings
from semantic_kernel.memory.null_memory import NullMemory


sk_client_large = MistralAIChatCompletion(
    ai_model_id="mistral-large-latest",
    api_key=os.environ["MISTRAL_API_KEY"]
)
sk_client_medium = MistralAIChatCompletion(
    ai_model_id="mistral-medium-latest",
    api_key=os.environ["MISTRAL_API_KEY"]
)
# settings = MistralAIChatPromptExecutionSettings(
#     temperature=0.5,
# )

large_model_client = SKChatCompletionAdapter(
    sk_client_large,
    kernel=Kernel(memory=NullMemory()),
    # prompt_settings=settings,
    model_info={
        "function_calling": True,
        "json_output": True,
        "vision": False,
        "family": ModelFamily.UNKNOWN,
        "structured_output": True,
    },
)

medium_model_client = SKChatCompletionAdapter(
    sk_client_medium,
    kernel=Kernel(memory=NullMemory()),
    # prompt_settings=settings,
    model_info={
        "function_calling": True,
        "json_output": True,
        "vision": False,
        "family": ModelFamily.UNKNOWN,
        "structured_output": True,
    },
)

## 7. Реализуем Tool для векторного поиска в Qdrant

In [ ]:
def search_mitre_techniques(input_query, vector_store, top_k: int = 2):
    """Search for relevant MITRE techniques based on query text"""
    try:
        search_query_result = vector_store.similarity_search_with_score(
            input_query, k=top_k
        )
        logger.debug(f"Retrieved {len(search_query_result)} documents")
        results = search_query_result  # (документ, score)
            
        logger.debug(f"Found {len(results)} relevant MITRE techniques for query")
        return results
    except Exception as e:
        logger.error(f"Error searching MITRE techniques: {str(e)}")
        return []

def search_mitre_techniques_tool(
    event_description: Annotated[str, "A natural language description of the suspicious SIEM event or observed behavior."]
) -> Annotated[str, "A JSON string containing a list of potentially relevant MITRE ATT&CK techniques with IDs, names, and relevance scores."]:
    """
    Performs a vector search in the Qdrant database to find relevant MITRE ATT&CK techniques
    based on the description of a SIEM event or observed behavior.
    """
    try:

        logger.debug("--- Calling Qdrant Search Tool ---")
        logger.debug("Searching for: {event_description}")
        
        search_results = search_mitre_techniques(event_description, vector_store, top_k=2)

        logger.debug(f"Qdrant Search Results: {search_results}")
        logger.debug("--- End Qdrant Search Tool ---")
        if not search_results:
            return "No relevant MITRE techniques found in the vector store for the given description."
        
        results = []
        for document, score in search_results:
            results.append({'search_content': document.page_content, 'search_metadata': document.metadata, 'search_hit_score': score})
        
        # RateLimits для Mistral слишком хардовый
        time.sleep(2)
        return json.dumps(results)

    except Exception as e:
        logger.error(f"Error during Qdrant search: {e}")
        logger.info(f"--- End Qdrant Search Tool ---")
        return f"Error performing Qdrant search: {str(e)}"


In [22]:
await search_mitre_techniques_tool(event_description='пользователь залогинился в аккаунт')

2025-05-19 22:17:32.901 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:search_mitre_techniques_tool:25 - --- Calling Qdrant Search Tool ---
2025-05-19 22:17:32.902 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:search_mitre_techniques_tool:26 - Searching for: {event_description}


10


2025-05-19 22:17:33.277 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:search_mitre_techniques:7 - Retrieved 3 documents
2025-05-19 22:17:33.278 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:search_mitre_techniques:10 - Found 3 relevant MITRE techniques for query
2025-05-19 22:17:33.279 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:search_mitre_techniques_tool:30 - Qdrant Search Results: [(Document(metadata={'id': 'attack-pattern--8861073d-d1b8-4941-82ce-dce621d398f0', 'name': 'Cloud Services', 'kill_chain_phases': [{'kill_chain_name': 'mitre-attack', 'phase_name': 'lateral-movement'}], 'external_references': [{'source_name': 'mitre-attack', 'url': 'https://attack.mitre.org/techniques/T1021/007', 'external_id': 'T1021.007'}], '_id': 'ab32228f-68e0-4da9-ad8b-e74b9ff76482', '_collection_name': 'mitre_demo'}, page_content='Adversaries may log into accessible cloud services within a compromised environment using [Valid Accounts](https://attack.mitre.org/techniques/T1078) that are synchronized with or federated to on-premis

'[{"search_content": "Adversaries may log into accessible cloud services within a compromised environment using [Valid Accounts](https://attack.mitre.org/techniques/T1078) that are synchronized with or federated to on-premises user identities. The adversary may then perform management actions or access cloud-hosted resources as the logged-on user. \\n\\nMany enterprises federate centrally managed user identities to cloud services, allowing users to login with their domain credentials in order to access the cloud control plane. Similarly, adversaries may connect to available cloud services through the web console or through the cloud command line interface (CLI) (e.g., [Cloud API](https://attack.mitre.org/techniques/T1059/009)), using commands such as <code>Connect-AZAccount</code> for Azure PowerShell, <code>Connect-MgGraph</code> for Microsoft Graph PowerShell, and <code>gcloud auth login</code> for the Google Cloud CLI.", "search_metadata": {"id": "attack-pattern--8861073d-d1b8-4941-

## 7.2 Реализуем Tool для хождения в VT

In [8]:
import vt
import vt.error 
import re


vt_client = vt.Client(os.getenv("VT_API_KEY"))

async def resolve_ip_virustotal_tool(
    ip_address: Annotated[str, "The public IPv4 or IPv6 address to query VirusTotal for."]
) -> Annotated[str, "A JSON string summarizing the VirusTotal analysis results for the IP address."]:
    """
    Queries the VirusTotal API for reputation information about a given IP address.
    """
    logger.debug("--- Calling VT IP resolve Tool ---")
    logger.info(f"Resolving IP: {ip_address}")
    if not vt_client:
        logger.debug("--- VT Client not available. Skipping resolution. ---")
        return json.dumps({"ip_address": ip_address, "error": "VirusTotal API key not configured."})

    try:
        # Basic check for private IPs
        if re.match(r'^(10\.|172\.(1[6-9]|2[0-9]|3[01])\.|192\.168\.|127\.)', ip_address):
             logger.debug(f"--- Skipping private IP: {ip_address} ---")
             return json.dumps({"ip_address": ip_address, "status": "private", "message": "Skipped resolution for private IP address."})

        ip_object = vt_client.get_object(f"/ip_addresses/{ip_address}")
        stats = ip_object.last_analysis_stats

        result = {
            "ip_address": ip_address,
            "status": "found",
            "last_analysis_stats": {
                "malicious": stats.get('malicious', 0),
                "suspicious": stats.get('suspicious', 0),
                "harmless": stats.get('harmless', 0),
                "undetected": stats.get('undetected', 0),
            },
            "reputation": ip_object.reputation,
            "owner": ip_object.get('as_owner', 'N/A'),
            "country": ip_object.get('country', 'N/A')
        }
        logger.debug(f"VT IP Result: {result}")
        logger.debug("--- End VT IP resolve Tool ---")

        time.sleep(2) # Снова Рейт лимиты
        return json.dumps(result)

    except vt.error.APIError as e:
        logger.error(f"VT API Error for IP {ip_address}: {e}")
        logger.debug("--- End VT IP resolve Tool ---")
        if "NotFoundError" in str(e):
             return json.dumps({"ip_address": ip_address, "status": "not_found", "message": "IP address not found in VirusTotal."})
        else:
             return json.dumps({"ip_address": ip_address, "status": "error", "message": f"VT API Error: {str(e)}"})
    except Exception as e:
        logger.error(f"Error during VirusTotal IP resolution for {ip_address}: {e}")
        logger.debug("--- End VT IP resolve Tool ---")
        return json.dumps({"ip_address": ip_address, "status": "error", "message": f"Unexpected error: {str(e)}"})


In [ ]:
await resolve_ip_virustotal_tool('3.59.19.3')

2025-05-19 22:57:23.040 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:resolve_domain_virustotal_tool:65 - --- Calling VT Domain resolve Tool ---
2025-05-19 22:57:23.040 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:resolve_domain_virustotal_tool:66 - Resolving Domain: google.com
2025-05-19 22:57:23.509 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:resolve_domain_virustotal_tool:93 - VT Domain Result: {'domain_name': 'google.com', 'status': 'found', 'last_analysis_stats': {'malicious': 0, 'suspicious': 0, 'harmless': 67, 'undetected': 27}, 'reputation': 622, 'categories': {'alphaMountain.ai': 'Virtual Meetings (alphaMountain.ai)', 'Sophos': 'information technology', 'Forcepoint ThreatSeeker': 'web collaboration'}, 'registrar': 'MarkMonitor Inc.', 'creation_date': '874306800'}
2025-05-19 22:57:23.509 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:resolve_domain_virustotal_tool:94 - --- End VT Domain resolve Tool ---


{'domain_name': 'google.com',
 'status': 'found',
 'last_analysis_stats': {'malicious': 0,
  'suspicious': 0,
  'harmless': 67,
  'undetected': 27},
 'reputation': 622,
 'categories': {'alphaMountain.ai': 'Virtual Meetings (alphaMountain.ai)', 'Sophos': 'information technology', 'Forcepoint ThreatSeeker': 'web collaboration'},
 'registrar': 'MarkMonitor Inc.',
 'creation_date': '874306800'}

In [9]:
from autogen_agentchat.agents import AssistantAgent


# MITRE Mapper agent
mitre_mapper = AssistantAgent(
    name="mitre_expert_mapper",
    model_client=large_model_client,
    tools=[search_mitre_techniques_tool],
    reflect_on_tool_use=True,
    system_message="""You are a cybersecurity expert specializing in MITRE ATT&CK framework."""
)

# VT Resolver agent
vt_resolver = AssistantAgent(
    name="vt_resolver",
    model_client=large_model_client,
    tools=[resolve_ip_virustotal_tool],
    reflect_on_tool_use=True,
    system_message="""You are a cybersecurity threat intelligence analyst."""
)

# Structured Output Parser agent
parser_agent = AssistantAgent(
    name="structured_output_parser",
    model_client=medium_model_client,
    system_message="""You are an expert in parsing and structuring security event data. 
    Your task is to take the analyses from other agents and structure them into a consistent JSON format. 
    Ensure that all relevant information is preserved and properly organized."""
)

# Summarization agent
summary_agent = AssistantAgent(
    name="summarization_chain",
    model_client=medium_model_client,
    system_message="""You are a security analyst tasked with summarizing security incidents. 
    Based on the structured data provided, create a concise but comprehensive summary of the security events, 
    highlighting potential threats, MITRE ATT&CK techniques identified, and indicators of compromise. 
    Focus on the most significant findings and potential impact."""
)

# Translation agent
translation_agent = AssistantAgent(
    name="translation",
    model_client=medium_model_client,
    system_message="""Ты эксперт переводчик. Твоя задача перевести полученный отчет на русский язык."""
)

## 8. Process Events with Agents

In [12]:
async def process_event_with_agents(event):
    """Process a single event with the agent workflow"""
    try:
        # Формируем описание события
        event_description = f"""Event ID: {event.get('id', 'Unknown')}
Time: {event.get('time', 'Unknown')}
Action: {event.get('action', 'Unknown')}
Status: {event.get('status', 'Unknown')}
Source IP: {event.get('src_ip', 'Unknown')}
Destination Host: {event.get('dst_hostname', 'Unknown')}
Event Source: {event.get('event_src_title', 'Unknown')}
Category: {event.get('category_high', 'Unknown')} / {event.get('category_generic', 'Unknown')} / {event.get('category_low', 'Unknown')}
Body: {event.get('body', 'Unknown')}"""
        
        # Шаг 1: Агент по Митре
        logger.info(f"Generating MITRE analysis for event {event.get('id', 'Unknown')}")
        mitre_analysis = await mitre_mapper.run(task= \
            f"""Your role is to analyze SIEM events and map them to relevant MITRE ATT&CK techniques.
You MUST use the 'search_mitre_techniques' tool, providing it with a brief description of the observed behavior from the SIEM event, to find potential techniques.
Analyze the results returned by the tool.

After executing the tool and getting the result:
1. Map possible TTP information to SIEM normalized event.
2. Provide actionable remediation steps tailored to the alert.
3. Cross-reference historical patterns and related alerts.
4. Recommend external resources for deeper understanding.

Ensure that:
- TTPs are tagged with the tactic, technique name, and technique ID.
- Remediation steps are specific and actionable.
- Historical data includes related alerts and notable trends.
- External links are relevant to the observed behavior.

If the tool returns an error or no results, state that clearly.

SIEM Event:
{event_description}

You MUST use the 'search_mitre_techniques' tool, providing it with a brief description of the observed behavior from the SIEM event, to find potential techniques.""")
        time.sleep(20)
        
        # Шаг 2: Агент по VT
        logger.info(f"Generating VT analysis for event {event.get('id', 'Unknown')}")
        vt_analysis = await vt_resolver.run(task= \
            f"""Your role is to analyze SIEM event descriptions to identify potential Indicators of Compromise (IoCs), specifically PUBLIC IP addresses and domain names.
You have two tools available: 'resolve_ip_virustotal' and 'resolve_domain_virustotal'.

Your process MUST be:
1. Read the provided SIEM event description carefully.
2. Identify any potential PUBLIC IPv4/IPv6 addresses or domain names within the text. Ignore private IPs (like 10.x.x.x, 192.168.x.x, 172.16.x.x-172.31.x.x, 127.x.x.x).
3. For EACH distinct public IP address identified, call the 'resolve_ip_virustotal' tool exactly once, passing the IP address as the argument.
4. For EACH distinct domain name identified, call the 'resolve_domain_virustotal' tool exactly once, passing the domain name as the argument.
5. If no public IPs or domains are found in the text, state that clearly.
6. Once all necessary tool calls are complete (or if none were needed), synthesize the results from the tools (or the lack thereof) into a single, structured JSON report.
   The report should list each checked IoC and the summary of findings from VirusTotal (malicious/suspicious counts, status like 'found', 'not_found', 'private', or 'error'). Include the VT link if available.
Do NOT guess IoCs. Only use the tools for IoCs explicitly mentioned in the input text. If a tool returns an error or 'not_found', report that in your final summary.
            
SIEM Event:
{event_description}""")
        time.sleep(20)
        
        # Шаг 3: Формируем json через агента
        logger.info(f"Parsing outputs for event {event.get('id', 'Unknown')}")
        structured_output = await parser_agent.run(task= \
            f"""Parse the following security analyses into a structured JSON format.

MITRE Analysis:
{mitre_analysis}

VT Resolver Analysis:
{vt_analysis}

Original Event:
{event_description}

Respond with a single, well-structured JSON object that integrates all this information.
"""
        )
        time.sleep(20)
        
        return {
            'event': event,
            'mitre_analysis': mitre_analysis,
            'vt_analysis': vt_analysis,
            'structured_output': structured_output,
            'status': 'success'
        }
    except Exception as e:
        logger.error(f"Error processing event: {str(e)}")
        logger.error(traceback.format_exc())
        return {
            'event': event,
            'error': str(e),
            'status': 'failed'
        }

# Process events
async def process_events_batch(events, sample_limit_size=3):
    """Process a batch of events with agents"""
    results = []
    total = len(events)
    
    logger.info(f"Processing {sample_limit_size} events from total of {total}...")
    
    for i, event in enumerate(events[:sample_limit_size]):
        try:
            logger.info(f"Processing event {i+1}/{min(total, sample_limit_size)}: {event.get('id', 'Unknown')}")
            start_time = time.time()
            result = await process_event_with_agents(event)
            elapsed = time.time() - start_time
            logger.info(f"Completed in {elapsed:.2f} seconds")
            results.append(result)
            logger.info(f"Processed event: {event.get('id', 'Unknown')} ({i+1}/{min(total, sample_limit_size)})")
        except Exception as e:
            logger.error(f"Error processing event {event.get('id', 'Unknown')}: {str(e)}")
            results.append({
                'event': event,
                'error': str(e),
                'status': 'failed'
            })
    
    return results

# Process a sample of events
sample_size = min(3, len(processed_events))  # Process up to 3 events as a sample
results = await process_events_batch(processed_events, sample_limit_size=sample_size)

2025-05-21 16:46:32.474 | INFO     | SIEM_AUTOGEN_ANALYSIS:process_events_batch:103 - Processing 7 events in limited size of 3...
2025-05-21 16:46:32.475 | INFO     | SIEM_AUTOGEN_ANALYSIS:process_events_batch:107 - Processing event 1/3: PT_Microsoft_Windows_eventlog_Sysmon_3_Network_connection
2025-05-21 16:46:32.476 | INFO     | SIEM_AUTOGEN_ANALYSIS:process_event_with_agents:16 - Generating MITRE analysis for event PT_Microsoft_Windows_eventlog_Sysmon_3_Network_connection
2025-05-21 16:46:35.409 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:search_mitre_techniques_tool:25 - --- Calling Qdrant Search Tool ---
2025-05-21 16:46:35.418 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:search_mitre_techniques_tool:26 - Searching for: {event_description}
2025-05-21 16:46:36.169 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:search_mitre_techniques:7 - Retrieved 3 documents
2025-05-21 16:46:36.170 | DEBUG    | SIEM_AUTOGEN_ANALYSIS:search_mitre_techniques:10 - Found 3 relevant MITRE techniques for query
2025-05-21 16:46:36.171

In [13]:
print(results[0]['structured_output'].messages[-1].content)

```json
{
  "event_id": "PT_Microsoft_Windows_eventlog_Sysmon_3_Network_connection",
  "time": "2024-11-28T14:40:05.5180000Z",
  "action": "detect",
  "status": "success",
  "source_ip": "10.156.92.66",
  "destination_ip": "3.5.140.3",
  "destination_port": 8888,
  "process": "C:\\\\Users\\\\Public\\\\wp.exe",
  "user": "NT AUTHORITY\\\\SYSTEM",
  "host": "wperry.dev.it.stf",
  "mitre_analysis": {
    "mapped_ttps": [
      {
        "tactic": "Discovery",
        "technique_name": "System Network Configuration Discovery",
        "technique_id": "T1016",
        "description": "Adversaries may look for details about the network configuration and settings, such as IP and/or MAC addresses, of systems they access or through information discovery of remote systems.",
        "external_reference": "https://attack.mitre.org/techniques/T1016"
      },
      {
        "tactic": "Discovery",
        "technique_name": "System Network Connections Discovery",
        "technique_id": "T1049",
    

## 9. Предварительно аггрегируем полученые результаты 

In [14]:
def merge_and_aggregate_results(results):
    """Merge and aggregate the results from the agent processing"""
    structured_outputs = []
    
    for result in results:
        if 'error' in result:
            continue
        structured_outputs.append(result['structured_output'].messages[-1].content)
    
    techniques = []
    for output in structured_outputs:
        # Попробуем просто достать TTP
        if isinstance(output, str) and 'technique' in output.lower():
            found = re.findall(r'T\d{4}(?:\.\d{3})?', output)
            techniques.extend(found)
    
    unique_techniques = list(set(techniques))
    
    merged_data = {
        'total_events': len(results),
        'identified_techniques': unique_techniques,
        'technique_count': len(unique_techniques),
        'processed_results': structured_outputs
    }
    
    return merged_data

merged_results = merge_and_aggregate_results(results)

## 10. Генерируем отчет-саммари

In [15]:
async def generate_summary(merged_results):
    """Generate a summary of the security analysis"""
    try:
        merged_results_str = json.dumps(merged_results, indent=2)
        
        logger.info("Generating summary...")
        summary = await summary_agent.run(task = \
            f"""Based on the following security analysis results, generate a comprehensive security summary.
Focus on the most significant findings, potential threats, MITRE ATT&CK techniques identified, 
and indicators of compromise. Provide an assessment of the overall security situation and any
recommended actions.

Analysis Results:
{merged_results_str}

Your summary should be well-structured and professional, suitable for a security operations team.
Include the following sections:
1. Executive Summary (one paragraph overview)
2. Key Findings (bullet points of the most important findings)
3. Technical Details (details about techniques, IoCs, etc.)
4. Recommendations (what should be done based on this analysis)"""
        )

        time.sleep(1)
        rus_summary = await translation_agent.run(task = f"Твоя задача перевести отчет на русский язык, сохраняя markdown форматирование:\n\n {summary}")
        
        return rus_summary
    except Exception as e:
        logger.error(f"Error generating summary: {str(e)}")
        return f"Error summary: {str(e)}"

rus_summary = await generate_summary(merged_results)


2025-05-21 16:54:24.050 | INFO     | SIEM_AUTOGEN_ANALYSIS:generate_summary:6 - Generating security summary...


In [16]:
from IPython.display import Markdown, display
import datetime

def display_summary(summary):
    """Display the summary as formatted markdown in the notebook"""
    display(Markdown(f"""

{summary.messages[-1].content}

---
*Generated on: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*
    """))

try:
    display_summary(rus_summary)
except Exception as e:
    logger.error(f"Could not display formatted summary: {str(e)}")



### Исполнительное резюме

Анализ безопасности предоставленных событий выявляет серию подозрительных действий на хосте `wperry.dev.it.stf`, что указывает на потенциальные злонамеренные действия. События включают сетевые подключения, создание процессов и создание файлов, все из которых связаны с процессами, работающими в контексте `NT AUTHORITY\SYSTEM`. Выявленные техники MITRE ATT&CK предполагают действия, связанные с обнаружением, уклонением от защиты, повышением привилегий и устойчивостью. Наиболее значимые выводы включают выполнение подозрительных процессов из общедоступного каталога, потенциальное злоупотребление Windows Management Instrumentation (WMI) для устойчивости и создание временных дамп-файлов, которые могут использоваться для доступа к учетным данным.

### Ключевые выводы

- **Подозрительное сетевое подключение**: Сетевое подключение было инициировано процессом `C:\Users\Public\wp.exe` к внешнему IP-адресу `3.5.140.3` на порту `8888`.
- **Подозрительное создание процесса**: Процесс `C:\Windows\System32\WerFault.exe` был создан родительским процессом `C:\Users\Public\msf.exe`, оба работают под `NT AUTHORITY\SYSTEM`.
- **Подозрительное создание файла**: Процесс `C:\WINDOWS\system32\WerFault.exe` создал временный дамп-файл `C:\ProgramData\Microsoft\Windows\WER\Temp\WER3716.tmp.dmp`.
- **Выявленные техники MITRE ATT&CK**:
  - **Обнаружение**: `T1016` (Обнаружение конфигурации сети системы), `T1049` (Обнаружение сетевых подключений системы)
  - **Уклонение от защиты**: `T1218.007` (Msiexec), `T1027.011` (Безфайловое хранилище)
  - **Доступ к учетным данным**: `T1003.001` (Память LSASS)
  - **Повышение привилегий/Устойчивость**: `T1546.003` (Подписка на события Windows Management Instrumentation), `T1543` (Создание или изменение системного процесса)

### Технические детали

#### Событие 1: Сетевое подключение
- **Идентификатор события**: `PT_Microsoft_Windows_eventlog_Sysmon_3_Network_connection`
- **Время**: `2024-11-28T14:40:05.5180000Z`
- **Источник IP**: `10.156.92.66`
- **Назначение IP**: `3.5.140.3`
- **Порт назначения**: `8888`
- **Процесс**: `C:\Users\Public\wp.exe`
- **Пользователь**: `NT AUTHORITY\SYSTEM`
- **Хост**: `wperry.dev.it.stf`
- **Техники MITRE**: `T1016`, `T1049`

#### Событие 2: Создание процесса
- **Идентификатор события**: `PT_Microsoft_Windows_eventlog_Sysmon_1_Process_creation`
- **Время**: `2024-11-28T14:14:11.0460000Z`
- **Процесс**: `C:\Windows\System32\WerFault.exe`
- **Родительский процесс**: `C:\Users\Public\msf.exe`
- **Пользователь**: `NT AUTHORITY\SYSTEM`
- **Хост**: `wperry.dev.it.stf`
- **Техники MITRE**: `T1546.003`, `T1218.007`, `T1543`

#### Событие 3: Создание файла
- **Идентификатор события**: `PT_Microsoft_Windows_eventlog_Sysmon_11_File_create`
- **Время**: `2024-11-28T14:14:11.2000000Z`
- **Процесс**: `C:\WINDOWS\system32\WerFault.exe`
- **Целевой файл**: `C:\ProgramData\Microsoft\Windows\WER\Temp\WER3716.tmp.dmp`
- **Пользователь**: `NT AUTHORITY\SYSTEM`
- **Хост**: `wperry.dev.it.stf`
- **Техники MITRE**: `T1027.011`, `T1003.001`, `T1546.003`

### Рекомендации

1. **Расследование подозрительных процессов**:
   - Проверить легитимность процессов `C:\Users\Public\wp.exe` и `C:\Users\Public\msf.exe`.
   - Использовать инструменты исследования процессов, такие как Sysinternals Process Explorer, для проверки деталей процессов и их родительско-дочерних отношений.

2. **Проверка сетевых подключений**:
   - Проанализировать сетевые подключения, инициированные подозрительными процессами.
   - Использовать `netstat` или аналогичные инструменты для списка активных подключений и проверки, ожидается ли подключение к `3.5.140.3:8888`.

3. **Проверка контекста пользователя**:
   - Расследовать, почему процессы работают под `NT AUTHORITY\SYSTEM`.
   - Проверить системные журналы и конфигурации, чтобы понять, почему процессы в общедоступном каталоге работают с привилегиями системы.

4. **Блокировка подозрительных IP-адресов**:
   - Заблокировать IP-адрес назначения `3.5.140.3`, если он не распознается как легитимный сервис.
   - Обновить правила брандмауэра для блокировки исходящих подключений к подозрительному IP-адресу.

5. **Усиление мониторинга**:
   - Увеличить мониторинг на хосте `wperry.dev.it.stf`.
   - Внедрить дополнительное ведение журналов и оповещения для любых необычных сетевых активностей, выполнений процессов или созданий файлов.

6. **Проверка подписок WMI**:
   - Проверить любые необычные подписки на события WMI, которые могут использоваться для устойчивости.
   - Использовать скрипты PowerShell или инструменты WMI для списка и проверки активных подписок WMI.

Реализуя эти рекомендации, команда операций безопасности может снизить потенциальные угрозы и усилить общую безопасность пораженного хоста.

---
*Generated on: 2025-05-21 16:57:16*
    